# The micro web application framework Flask

## What is Flask?

Flask is a micro web application framework.  

You can use Flask to:
* Build a static website (blog, private homepage)
* Build dynamic websites
* Build an API server (application interface)

Some features:
* Development server and debugger
* Integrated support for unit testing
* Unicode-based
* Open-source
* Flask is used by some larger companies such as Pinterest and LinkedIn, giving it some credibility.


Flask comes with an HTTP server that can serve static or dynamically generated HTML documents - or any other tpyes of documents.

### Documentation

http://flask.pocoo.org/docs


### Alternative platforms
* Django (https://www.djangoproject.com)
* Tornado (http://www.tornadoweb.org/en/stable/)
* Ruby on Rails (http://rubyonrails.org) 

Flask has no database abstraction layer, no form validation or similar advanced features. This differentiates it from other web frameworks such as Ruby on Rails or Django.
However, you can use third-party extensions to obtain the additional functionality of more advanced web applications.

## Hello world in Flask

In [1]:
# File: my_webserver.py
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__=="__main__":
    app.run()

Run the flask server with:
```bash
python my_webserver.py
   * Running on http://localhost:5000/
```            

Then open http://localhost:5000/ in your web browser. To stop the server, hit `Control-C`.


## What happened?

```python
from flask import Flask
app = Flask(__name__)
```
An instance of the `Flask` class is our web application.
The first argument is needed so that Flask knows where to look for templates, static files, and so on.  If you are using a single module, `__name__` is always the correct value ([more info](http://flask.pocoo.org/docs/0.10/api/#flask.Flask)). 

```python
@app.route("/")
def hello():
    return "Hello World!"
```

We then use the route() decorator to tell Flask what URL should trigger our function. By default, Flask answers to GET requests, and the return value of the function is the answer of the `GET` request.

```python
if __name__ == "__main__":
    app.run()
```

Finally we use the `run()` function to run the local server with our application. 

**Note**: You need to restart the sever when changing your code.

**Debug mode**: You can use `app.run(debug=True)` to see more detailed error output. Also, in debug mode the server will reload itself on code changes. 

If you want your server to be accessible from your entire network, you need to launch the server with:
```python
app.run(host='0.0.0.0')
```
**Important**: This allows *anyone* in your network to access your webbrowser, which might be a severe security riks!

## Adding more URLs

We can serve additional URLs by adding new functions with the route decorator:

In [2]:
users = {"richard": "Richard Lee",
         "john": "John Smith"}

@app.route('/user')
def show_user_overview():
    users_str = "<br>".join(users.values())
    return '<h1>Our users</h1><br>{}'.format(users_str)

Matches only http://localhost:5000/user

In [3]:


@app.route('/user/<username>')
def show_user_profile(username):
    # show the user profile for that user
    return 'User %s' % username

Matches http://localhost:5000/user/NAME for any `NAME`

In [ ]:
app.run()

ERROR:__main__:Exception on /user [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/flask/app.py", line 1817, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python2.7/dist-packages/flask/app.py", line 1477, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python2.7/dist-packages/flask/app.py", line 1381, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python2.7/dist-packages/flask/app.py", line 1475, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python2.7/dist-packages/flask/app.py", line 1461, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-a2bb5d5ec187>", line 7, in show_user_overview
    return '<h1>Our users</h1><br>{}'.format(users_str)
NameError: global name 'users_str' is not defined


## Templates

So far our webserver only served simple textstrings, but no HTML documents. 

One solution would be to define the entire HTML string in the URL handler, e.g.:

```python
@app.route('/login')
def login():
    return '''
<html>
<header><title>The title</title></header>
<body>
Hello world
</body>
</html>    
    '''
```

However, Flask comes with a templating system that makes our life a lot easier:

In [17]:
from flask import render_template

@app.route('/post/')
@app.route('/post/<name>')
def post(name=None):
    return render_template('post.html', name=name)

Flask will look for templates in the `templates` folder.

In [18]:
%ls ./templates/post.html

./templates/post.html


```html
<!-- ./templates/post.html -->

<!doctype html>
<title>Hello from Flask</title>
{% if name %}
  <h1>Displaying blog post {{ name }}!</h1>
{% else %}
  <h1>No post name given!</h1>
{% endif %}
```

In [19]:
app.run()

## HTML form

Using the template, we can now create a HTML form with a `POST` requets

```html
<!-- ./templates/login.html -->

<!doctype html>
<title>Login</title>

{% if error %}
<p style="color:red">{{ error }}</p>
{% endif %}

<form action="handle_login" method="POST">
    Username:
    <br>
    <input type="text" name="username">
    <br>
    Password:
    <br>
    <input type="password" name="password">
    <br>
    <input type="submit" value="Submit">
</form>
```


In [ ]:
@app.route('/login')
def login():
    return render_template('login.html')

In [ ]:
app.run()

## Handling the `POST` request.

The form above sends a `POST` request to the `handle_login` URL. 

We can use 
```python
@app.route('/handle_login', methods=['POST'])     
```
to create a new Flask handler that accepts `POST` requests.

We can then use the 
```python
flask.request
```
module to access the data in the `POST` request (here the username and password that the user provided in the form).

In [ ]:
from flask import request
                                                                                                                                                                                                                                               
@app.route('/handle_login', methods=['POST'])                                                                                                                                                                                                  
def handle_login():           
    
    assert request.method == 'POST'   # Check that we are really in a POST request
    
    # Acces the form data:
    username = request.form["username"]
    password = request.form["password"]
    
    if username == "simon" and password == "safe":                                                                                                                                                             
        return "You are logged in Simon"                                                                                                                                                                                                       
    else:                                                                                                                                                                                                                                      
        error = "Invalid credentials"                                                                                                                                                                                                          
        return render_template("login.html", error=error)     

In [ ]:
app.run()